In [2]:
%pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=54d1ed867458cd4c0904ec24e5f9b31548c98f33dda15c697ebe20187c21072b
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
import wget

In [4]:
# ## wget: url의 파일들을 가져올 수 있는 기능
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

'ratings_train.txt'

In [5]:
import pandas as pd

In [6]:
df_naver_movie_comment=pd.read_csv('./ratings_train.txt', sep='\t')
df_naver_movie_comment[:3] # id: 게시판 number, document: 댓글, label: 긍정1 부정 0

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


## 머신러닝 의한 감성 분석
- 0. label: 긍정 1, 부정 0
    - 1인것은 긍정 array로, 0인것은 부정 array로 합친다.
- 1. 형태소 분석을 통해 품사 분류를 진행
- 2. vector 토큰화 (단어를 숫자화(OHE이용?))
- 3. 목표 변수 생성
- 4. 머신러닝 학습
- 5. 머신러닝 학습을 통한 예측


### 0. 긍정 1, 부정 0
- label이 0에 해당하는 것과 1에 해당하는 것을 나눠서 합친다.?
- 일단 0 따로 1따로 나눠봐.
- 나눈 다음에 positive, negative로 나누고 그걸 positive + negative로 합친다.

In [7]:
condition_neg=(df_naver_movie_comment['label'] == 0 )
df_movie_negative=df_naver_movie_comment[condition_neg]

In [9]:
doc_negative=df_movie_negative['document'][:1000].values

In [10]:
condition_pos=(df_naver_movie_comment['label'] == 1 )
df_movie_positive=df_naver_movie_comment[condition_pos]

In [11]:
doc_positive=df_movie_positive['document'][:1000].values

In [12]:
type(doc_negative), type(doc_positive)

(numpy.ndarray, numpy.ndarray)

In [13]:
doc_total=doc_positive + doc_negative

In [14]:
len(doc_total)

1000

### 형태소 분석을 통해 품사 분류를 진행


In [16]:
!pip install python-mecab-ko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 32.4 MB/s eta 0:00:00


In [17]:
from mecab import MeCab

In [18]:
mecab = MeCab()

문장 토큰화

In [ ]:
tokenized_comment = list()

for comment in doc_total :
    comment_morphs=mecab.morphs(comment)
    # print(comment_morphs) # ., ,, 한글자 불용어 빼기 len(word)>1 #list로 되어있으므로 이를 문장으로 만든다.
    # mecab.nouns(comment_morphs) : 하지 않는 이유는 문장으로 진행해야하기 때문
    # 원래는 .pos로 품사를 확인한 뒤 조사, 기호는 제외해야함.
    # 문장을 list로 되어있다면 이걸 looping 돌려서 제외시켜야함. tagging으로 형태소를 looping해야함.
    # print(' '.join(comment_morphs))
    # .append(' '.join(comment_morphs))
    tokenized_comment.append(comment_morphs)

tokenized_comment

In [23]:
mecab.nouns(doc_total[2])

['액션', '재미', '영화', '교도소', '이야기', '재미', '평점', '조정']

In [24]:
mecab.pos(doc_total[2])

[('액션', 'NNG'),
 ('이', 'JKS'),
 ('없', 'VA'),
 ('는데', 'EC'),
 ('도', 'JX'),
 ('재미', 'NNG'),
 ('있', 'VV'),
 ('는', 'ETM'),
 ('몇', 'MM'),
 ('안', 'MAG'),
 ('되', 'VV'),
 ('는', 'ETM'),
 ('영화', 'NNG'),
 ('교도소', 'NNG'),
 ('이야기', 'NNG'),
 ('구먼', 'VCP+EF'),
 ('.', 'SF'),
 ('.', 'SY'),
 ('솔직히', 'MAG'),
 ('재미', 'NNG'),
 ('는', 'JX'),
 ('없', 'VA'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('.', 'SY'),
 ('평점', 'NNG'),
 ('조정', 'NNG')]

In [35]:
nouns_list = list()
for word in doc_total:
    words = mecab.pos(word)
    filtered_words = [w for w, pos in words if (pos == 'MAG' or pos == 'NNG') and len(w) > 1]
    nouns_list.extend(filtered_words)

In [ ]:
# nouns_list = list()
# for word in doc_total :
#     words = mecab.nouns(word)
#     # 한글자 이상만 담기_chat에 물어봄
#     filtered_words = [w for w in words if len(w) > 1]
#     nouns_list.extend(filtered_words)
# nouns_list

In [36]:
combined_text = ' '.join(nouns_list)

In [37]:
combined_text

'초딩 영화 오버 연기 진짜 짜증 목소리 익살 연기 영화 너무나 액션 재미 영화 교도소 이야기 솔직히 재미 평점 조정 평점 너무 걸음마 초등 학교 학년 영화 별반 진짜 원작 긴장감 제대로 향수 자극 감성 절제 달인 반개 연기 생활 정말 납치 감금 반복 반복 드라마 가족 연기 사람 신문 기사 보다 자꾸 사람 횡단 보도 연기 매번 긴장 존중 진짜 내생 극장 영화 가장 감동 스토리 감동 어거지 사람 바스코 그냥 안달 레닌 표절 이해 갈수록 정말 깨알 캐스팅 내용 구성 깨알 음식 영화 만찬 만찬 이야기 음식 재미 음식 별로 풍경 구경 별로 약탈자 변명 절대 주제 중반 그냥 학생 선생 영화 절대 납득 가능 발연기 절대 영화 수작 말씀 쓰레기 시간 고추 기대 죄인 죄인 연기 대체 정신 장애 장애 센스 연출력 향수 관객 포스 위력 다시 검사 연기 정말 완전 명품 드라마 내용 완전 영화 아까 완전 별점 이리 고평 속지 시간 낭비 낭비 아직 드라마 최고 리얼리티 공감 캐릭터 정신 분노 조절 장애 초기 증상 툭하면 사람 물건 파손 조금 오바 초반 신선 갈수록 정신 상태 공감 불가 패션 열정 마이너스 보고 영화 수준 작가 정신 유령 영화 사랑 영화 이렇게 평가 고추 매운맛 저그 진호 단순 매력 영화 꼬마 원한 그냥 혼자 서리 최고 정말 방법 배우 발견 일탈 미소 음악 조금 평점 내내 그대로 예측 카리스마 악역 주인 누이 근친상간 소설 제일 드라마 제일 웃음 감독 토르 다크 월드 기본 선방 불알 당황 영화 중간 느낌 영화 사람 영혼 세상사 잠시 동화 영화 행복 일상 조금 이렇게 가슴 드라마 감동 자체 계속 전개 주인공 소극 모습 짜증 맨날 언제 나나 기대 전개 빨리빨리 예전 작품 캐릭터 에피소드 재탕 사골 내용 시청 아예 이제 중반 부작 연기력 초반 입도 너무 충격 기분 완전히 느낌 활력 너무나 지독 그저 일본인 정말 생각 노래 실력 진짜 망신 언제나 영화 유치 사랑 가슴속 감정 영화 정말 최고 졸작 사람 다큐 나라 현대사 단면 깊이 생각 사죄 바로 노력 보도 연맹 민간 학살 정도

### 벡터 토큰화
- countVectorizer = CountVectorizer()

In [38]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [39]:
countVectorizer = CountVectorizer()

In [41]:
countVectorizer.fit([combined_text])

CountVectorizer()

In [ ]:
모델 교육이 끝나면 새로운 영화 리뷰 댓글들을 넣고 예측 하기